In [78]:
import os
os.chdir("C:/Users/edohner/OneDrive - Lyric Opera of Chicago/Desktop/Python Learning/airflow_test_project")

In [79]:
import pandas as pd 
from api_library import SMConnect

sm_connection = SMConnect()

# Select just the 'pages' key, then the 'quesitons' key --> this returns a LIST of dicts
questions = sm_connection.survey_details_dict.get('pages')[0].get('questions')

INFO:root:sm_api_config.json found at C:\Users\edohner\OneDrive - Lyric Opera of Chicago\Desktop\Python Learning\airflow_test_project\api_library_package\api_library\sm_api\sm_api_config.json
INFO:root:Successfully loaded sm_api_config.json
INFO:root:sm_api_config.json found at C:\Users\edohner\OneDrive - Lyric Opera of Chicago\Desktop\Python Learning\airflow_test_project\api_library_package\api_library\sm_api\survey_details.json
INFO:root:Successfully loaded survey_details.json


In [80]:
# manually extracting the 'other' answer for q10
q_10_other_df = pd.DataFrame({  
    'other_id': [questions[9]['answers']['other']['id']],
    'other_text': [questions[9]['answers']['other']['text']]
    })

# assigning all q10 answer options to the q10 quesiton id and q10 queston text
q_10_other_df['question_id'] = questions[9]['id']
q_10_other_df['question_text'] = questions[9]['headings'][0]['heading']

# reordering columns of q_10_df for quesiton id is first
q_10_other_df = q_10_other_df[['question_id', 'question_text', 'other_id', 'other_text']]

In [81]:
# Create a for loop to handle all quesiton that have answer options (avoids open-ended response)

# Initalize list to hold each iteration through the for loop
# if we don't do this, the df we create will be overritten each time the loop cycles
df_list = []

# Here we loop throuhg all the quesiton dicts that qualify, applying the pd.json_normalize() function
for question in questions:
    if 'answers' in question:
        temp_df = pd.json_normalize(
            question, 
            record_path=['answers', 'choices'],
            meta=['id', 'headings'],
            meta_prefix='question_',
            record_prefix='answer_'
        )
        
        df_list.append(temp_df)
        
# combine all the dfs that accumulated in the df_list 
choices_df = pd.concat(df_list)

# Use a lambda function to pull out the quesiton title -- the heading -- nested in a dictoary
choices_df['question_text'] = choices_df['question_headings'].apply(lambda x: x['heading'])

# Select the columns of interest
choices_df = choices_df[['question_id', 'question_text', 'answer_id', 'answer_text']]

In [82]:
# Grab remaning questions: all open-ended responses. These DO NOT have an 'answer' dict included

# Initalize list to hold each iteration through the for loop
df_list = []

# Loop through all qualifying question dicts, applying pd.DataFrame function
for question in questions:
    if 'answers' not in question:
        temp_df = pd.json_normalize(
            question, 
            record_path=None
        )
    
    df_list.append(temp_df)
    
open_ended_answers_df = pd.concat(df_list)

# Use a lambda function to pull out the quesiton title -- the heading -- nested in a dictoary
open_ended_answers_df['question_text'] = open_ended_answers_df['headings'].apply(lambda x: x[0]['heading'])

# Rename id column
open_ended_answers_df.rename(columns={
    'id': 'question_id',
}, inplace=True)

# Select the columns of interest
open_ended_answers_df = open_ended_answers_df[['question_id', 'question_text']]

# There are some duplicated questions, drop them
open_ended_answers_df.drop_duplicates(inplace=True)

In [83]:

# combine all dfs 
all_question_answer_df = pd.concat([q_10_other_df, choices_df, open_ended_answers_df])

all_question_answer_df

,question_id,question_text,other_id,other_text,answer_id,answer_text
0,208013586,How would you describe yourself?,1492773544,Prefer to self describe,NaN,NaN
0,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773480,Not at all likely - 0
1,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773481,1
2,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773482,2
3,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773483,3
4,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773484,4
5,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773485,5
6,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773486,6
7,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773487,7
8,208013575,How likely is it that you would recommend thi...,NaN,NaN,1492773488,8


In [85]:
# finally upload the resulting table to our bigquert database, 
# under the data set 'staging', as this table will need to be 
# referenced when transforming data with DBT


# Init the BQConnection class 
# --> start up the bigquery client
# --> will then reference the 'project' attribute
from api_library import BQConnect
from google.cloud import bigquery
import logging

logging.basicConfig(level=logging.INFO)


bigquery_connection = BQConnect()

# define project_id, dataset_id and table_id
dataset_id = bigquery_connection.dataset_pipeline
table_id = f'{dataset_id}.raw_sm_questions'

# configure the specific table we're sending the df to in bigquery
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("question_id", "STRING"),
        bigquery.SchemaField("question_text", "STRING"),
        bigquery.SchemaField("answer_id", "STRING"),
        bigquery.SchemaField("answer_text", "STRING"),
        bigquery.SchemaField("other_id", "STRING"),
        bigquery.SchemaField("other_text", "STRING")
    ],
    write_disposition="WRITE_TRUNCATE",
)

# load the table to bigquery
load_job = bigquery_connection.client.load_table_from_dataframe(
    all_question_answer_df, table_id, job_config=job_config
)

# wait for the result
load_job.result()  

# Check job status and log
if load_job.state == 'DONE':
    if load_job.error_result:
        logging.error(f"Job failed with error: {load_job.error_result}")
    else:
        logging.info(f"Job completed successfully with {load_job.output_rows} rows loaded.")
else:
    logging.warning(f"Job state: {load_job.state}")

INFO:root:Job completed successfully with 49 rows loaded.


In [53]:
# # final cleaning for open_ended_answers_df
# open_ended_answers_df['question_id'] = open_ended_answers_df['question_id'].astype(int)


In [54]:
# # Configure load for open_ended_answers_df
# table_id = f'{dataset_id}.raw_sm_questions_open_ended_answers'

# # configure the specific table we're sending the df to in bigquery
# job_config = bigquery.LoadJobConfig(
#     schema=[
#         bigquery.SchemaField("question_id", "INTEGER"),
#         bigquery.SchemaField("question_text", "STRING")
#     ],
#     write_disposition="WRITE_TRUNCATE",
# )

# # load the table to bigquery
# load_job = bigquery_connection.client.load_table_from_dataframe(
#     open_ended_answers_df, table_id, job_config=job_config
# )

# # wait for the result
# load_job.result()  

# # Check job status and log
# if load_job.state == 'DONE':
#     if load_job.error_result:
#         logging.error(f"Job failed with error: {load_job.error_result}")
#     else:
#         logging.info(f"Job completed successfully with {load_job.output_rows} rows loaded.")
# else:
#     logging.warning(f"Job state: {load_job.state}")